# 导入库

In [ ]:
import statistics
import subprocess
import ctypes

import sten

import math
import numpy as np
import torch
from torch.profiler import profile, record_function, ProfilerActivity

from pathlib import Path

import timeit
import argparse
import sys
import time

from grouped_nmv_tensor import SrNMTensor, nm_vector_mask_sparsify

import spatha

/root/miniconda3/envs/end2end/lib/python3.12/site-packages/torch/_utils.py:842: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/root/miniconda3/envs/end2end/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/root/miniconda3/envs/end2end/lib/python3.12/inspect.py:586: UserWarning: The is_traceable field on torch.autograd.Function is deprecated and will be removed in PyTorch 2.4.
  value = getter(object, key)


In [ ]:
sys.argv = ['your_notebook_name', '-m', '16', '-n', '2', '-v', '64']

# 设置解析器
parser = argparse.ArgumentParser()

parser.add_argument('-m', type=int, default=16)
parser.add_argument('-n', type=int, default=2)
parser.add_argument('-v', type=int, default=128)

parser.add_argument('--profile', action='store_true', default=False)
parser.add_argument('--sparsetime', action='store_true', default=False)

# 解析参数
args = parser.parse_args()

# 提取参数
m          = args.m
n          = args.n
v          = args.v
profile    = args.profile
sparsetime = args.sparsetime

In [ ]:
# def vnm_pruning_torch(matrix, V, N, M,):
#     assert matrix.size(0) % V == 0, "矩阵的行数需要是 V 的倍数"
#     assert matrix.size(1) % M == 0, "矩阵的列数需要是 M 的倍数"
#     rows, cols = matrix.shape
#     pruned_matrix = torch.zeros_like(matrix)

#     # 遍历每个 VxM 块
#     for row_block in range(0, rows, V):
#         for col_block in range(0, cols, M):
#             # 在每个块中随机选择 4 列作为可能的非零元素列
#             possible_cols = torch.randperm(M)[:4] + col_block

#             # 对每一行，从这 4 列中随机选择 N 列填入非零元素
#             for v_row in range(V):
#                 selected_cols = possible_cols[torch.randperm(4)[:N]]
#                 pruned_matrix[row_block + v_row, selected_cols] = matrix[row_block + v_row, selected_cols]

#     return pruned_matrix



# # 随机初始化矩阵并转移到 GPU，使用半精度
# matrix = torch.rand(4096, 2048, device="cuda:0", dtype=torch.float16)

# pruned_matrix = vnm_pruning_torch(matrix, v, n, m)

# # 保存矩阵
# torch.save(pruned_matrix, "pruned_matrix.pt")

# 读取矩阵
pruned_matrix = torch.load("pruned_matrix.pt")
pruned_matrix = pruned_matrix.to("cuda:0")

In [ ]:
def calculate_mask_and_columns(tensor, V, N, M):
    rows, cols = tensor.shape
    mask = torch.zeros_like(tensor, dtype=torch.int32)
    columns = torch.zeros((rows // V, cols // M * 4), dtype=torch.int32)

    # 使用unfold提取所有VxM块
    blocks = tensor.unfold(0, V, V).unfold(1, M, M)

    for i in range(blocks.shape[0]):
        for j in range(blocks.shape[1]):
            block = blocks[i, j]
            nonzero_cols = block.nonzero(as_tuple=False)[:, 1]
            unique_cols = torch.unique(nonzero_cols)
            num_unique = min(len(unique_cols), 4)
            columns[i, j*4: j*4 + num_unique] = unique_cols[:num_unique]

            for idx in range(V):
                row_nonzero_cols = block[idx, :].nonzero(as_tuple=False).squeeze()
                mask[i*V + idx, j*M + row_nonzero_cols] = 1

    return mask, columns

In [ ]:
mask, columns = calculate_mask_and_columns(pruned_matrix, v, n, m)

# 输出结果查看（可选，根据需要调整输出量）
print("Mask:")
print(mask)
print("Columns:")
print(columns)

In [5]:
def calculate_mask_and_columns(tensor, V, N, M):
    rows, cols = tensor.shape
    mask = torch.zeros_like(tensor, dtype=torch.int32)  # 使用 int32 类型
    columns = torch.zeros((rows // V, cols // M * 4), dtype=torch.int32)  # 存储每个块中非零元素的列索引

    # 遍历每个 VxM 的块
    for row_block in range(0, rows, V):
        for col_block in range(0, cols, M):
            block = tensor[row_block:row_block + V, col_block:col_block + M]

            # 确定块中所有非零元素的列位置
            nonzero_cols = block.nonzero(as_tuple=False)[:, 1]  # 获取非零元素的列索引

            if len(nonzero_cols) > 0:
                # 从中提取前四个不重复的列索引
                unique_cols = torch.unique(nonzero_cols)
                num_unique = min(len(unique_cols), 4)  # 最多取四列
                
                # 更新 columns 矩阵
                columns[row_block // V, (col_block // M) * 4: (col_block // M) * 4 + num_unique] = unique_cols[:num_unique]
                
                # 设置 mask
                for idx in range(V):  # 对于块中的每一行
                    row_nonzero_cols = block[idx, :].nonzero(as_tuple=False).squeeze()
                    mask[row_block + idx, col_block + row_nonzero_cols] = 1

    return mask, columns

In [6]:
mask, columns = calculate_mask_and_columns(pruned_matrix, v, n, m)

# 输出结果查看（可选，根据需要调整输出量）
print("Mask:")
print(mask)
print("Columns:")
print(columns)

Mask:
tensor([[0, 1, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int32)
Columns:
tensor([[ 1, 10, 14,  ...,  7,  8, 14],
        [ 0,  3,  4,  ...,  3, 12, 13],
        [ 0,  3, 11,  ...,  8, 11, 15],
        ...,
        [ 4,  5,  7,  ...,  2,  4, 11],
        [ 0,  1,  4,  ...,  5, 10, 11],
        [ 0,  5, 14,  ...,  8, 11, 12]], dtype=torch.int32)


In [7]:
class NMVectorSparsifier:
    def __init__(self, n, m, tileM, mask=None, columns=None):
        self.n = n
        self.m = m
        self.tileM = tileM
        self.mask = mask
        self.columns = columns

    def __call__(self, tensor):
        if self.mask is None or self.columns is None:
            # 未提供mask和columns，需要计算它们
            mask, columns = calculate_mask_and_columns(tensor, self.tileM, self.n, self.m)

        else:
            # 使用提供的mask和columns
            mask = self.mask
            columns = self.columns
        
        # 使用sten库创建一个稀疏张量包装器，这个稀疏张量基于之前创建的mask和columns
        sparse_mtx = sten.SparseTensorWrapper.wrapped_from_dense(
            SrNMTensor(self.n, self.m, self.tileM, tensor, mask, columns, tensor.device),
            tensor,
            grad_fmt,
        )
        
        return sparse_mtx

In [8]:
w = NMVectorSparsifier(n, m, v, mask, columns)(pruned_matrix).wrapped_tensor

: 

In [9]:
values = torch.nn.Parameter(w.values)
values

Parameter containing:
tensor([0.4963, 0.0885, 0.2386,  ..., 0.6001, 0.4131, 0.2450], device='cuda:0',
       dtype=torch.float16, requires_grad=True)

In [10]:
columns = w.columns
columns

tensor([[0, 1, 2,  ..., 1, 2, 3],
        [0, 1, 2,  ..., 1, 2, 3],
        [0, 1, 2,  ..., 1, 2, 3],
        ...,
        [0, 1, 2,  ..., 1, 2, 3],
        [0, 1, 2,  ..., 1, 2, 3],
        [0, 1, 2,  ..., 1, 2, 3]], device='cuda:0', dtype=torch.int32)

In [4]:
def sparse_dense_mul_dispatch(sparse_values, sparse_indices, sparse_metadata, dense, nrows_sp, ncols_sp, ncols_d, m, n, v, nnz, bias):

    dense_ = dense.contiguous()

    output = spatha.spmm(sparse_metadata,  # metadata
                          sparse_indices,   # indices
                          sparse_values,    # values
                          dense_,           # rhs_matrix
                          bias,
                          nrows_sp,         # A_num_rows
                          ncols_sp,         # A_num_cols
                          ncols_d,          # B_num_cols
                          v,                # vec_length
                          n,                # n
                          m,                # m
                          nnz,              # nnz
                          0,                # seed
                          32,               # mbrow
                          4                 # brow
                          )

    return output

In [5]:
class SrnmSpmm(torch.nn.Module):
    def __init__(self, original: torch.nn.Linear):
        super().__init__()
        self.bias = original.bias

        # Convert weights from original module to SrNM
        w = NMVectorSparsifier(n, m, v)(original.weight).wrapped_tensor

        self.values = torch.nn.Parameter(w.values)
        #self.columns = self.register_buffer('columns', w.columns)
        self.columns = w.columns
        self.metadata = w.metadata

        self.nrows_sp = w.nrows
        self.ncols_sp = w.ncols
        self.nnz      = w.nnz

    def forward(self, input):

        flattened_input = torch.flatten(input, start_dim=0, end_dim=-2)

        ncols_d  = flattened_input.T.shape[1]
        DM, _    = flattened_input.shape

        output = sparse_dense_mul_dispatch(self.values, self.columns, self.metadata, flattened_input.T, self.nrows_sp, self.ncols_sp,
                                           ncols_d, m, n, v, self.nnz, self.bias)
        output = output.reshape((*input.shape[0:-1], -1))[..., :DM]
        return output

In [6]:
def report_time(name, data, number):
    for d in data:
        time_ms = d / number * 1000
        #print(f'n {n} m {m} format {name} time_ms {time_ms:.3f}')
    ds = [(d / number * 1000) for d in data]
    mean = statistics.mean(ds)
    median = statistics.median(ds)
    std = statistics.stdev(ds)

    if name == "n:m":
        cfg = str(n)+","+str(m)+","
    else:
        cfg = "0,0,"
    print(
        "0,"+cfg+str(v)+","+str(mean)+","+str(median)+","+str(std)+","+str(len(ds))
    )

In [7]:
def linear_to_spmm(mod, weights_to_sparsify):
    if isinstance(mod, torch.nn.Linear):
        return SrnmSpmm(mod)

    for name, m in mod.named_children():
        if isinstance(m, SrnmSpmm):
            continue
        if isinstance(m, torch.nn.Linear):
            setattr(mod, name, SrnmSpmm(m))
        elif m is not mod:
            linear_to_spmm(m, weights_to_sparsify)

    return mod

In [8]:
def transformer_encoder_layer_prototype(num_repeats, number):
    # 加载原始的BERT大模型
    model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-large-uncased')

    # 加载第二个BERT大模型，并转移到CUDA设备上，使用半精度浮点数(half precision)进行计算以提高性能
    model2 = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-large-uncased').to(device='cuda:0').half()

    # 生成一个随机整数输入，大小为32x512，用于模型输入
    input = torch.randint(low=0, high=100, size=(32, 512))

    # 生成一个列表，包含模型中所有线性层（Linear layers）的权重，这些权重将被转换为稀疏格式
    weights_to_sparsify = [
        module
        for module_name, module in model.named_modules()
        if (
            isinstance(module, torch.nn.modules.linear.Linear)
            and "encoder.layer" in module_name
        )
    ]

    # 把模型和输入都转移到CUDA设备上，并使用半精度浮点数进行处理
    model = model.to(device='cuda:0').half()
    input = input.to(device='cuda:0')

    # 将选定的权重转换为稀疏格式，并创建一个新的稀疏模型
    sparse_model = linear_to_spmm(model, weights_to_sparsify)

    # 执行模型一次，通常用于预热缓存
    output = sparse_model(input)

    # 如果命令行参数指定进行性能分析
    if args.profile:
        # 开启一个性能分析会话，记录CPU和CUDA的活动
        with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, with_stack=True) as prof:
            with record_function("model_inference"):
                # 执行模型推理，记录相关数据
                output = sparse_model(input)
        # 导出分析数据
        prof.export_stacks("/tmp/profiler_stacks.txt", "self_cuda_time_total")
        prof.export_chrome_trace("trace_sparse.json")
        print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=15))

        # 重复上述过程，但是这次使用密集模型
        with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, with_stack=True) as prof:
            with record_function("model_inference"):
                output = model2(input)
        prof.export_stacks("/tmp/profiler_stacks_dense.txt", "self_cuda_time_total")
        prof.export_chrome_trace("trace_dense.json")
        print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=15))
        exit()

    # 使用timeit库，重复执行密集模型10次作为预热，然后进行正式的性能测试
    timeit.repeat('output = model2(input)', repeat=10, number=number, globals=locals())
    dense_times = timeit.repeat('output = model2(input)', repeat=num_repeats, number=number, globals=locals())
    report_time('dense', dense_times, number)

    # 对稀疏模型进行同样的预热和性能测试
    timeit.repeat('output = sparse_model(input)', repeat=10, number=number, globals=locals())
    sparse_times = timeit.repeat('output = sparse_model(input)', repeat=num_repeats, number=number, globals=locals())
    report_time('n:m', sparse_times, number)

In [9]:
torch. set_grad_enabled(False)

In [10]:
transformer_encoder_layer_prototype(num_repeats=30, number=1)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


0,0,0,64,28.492342207270365,28.310824069194496,0.7472337186168171,30
0,2,8,64,56.45544687286019,56.43744091503322,0.9001977038644426,30
